# M1 - 130 - Compute metrics

Using by levels scores computed during the fine-tuning :
* compute evaluation on joint-labels
* compute evaluation on all entities

In [52]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

0


In [53]:
import torch

print(torch.version.cuda)
print(torch.cuda.is_available())

11.6
True


In [54]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_1')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m1_independant_ner_layers', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages', '/lrde/home2/stual/.cache/huggingface/modules']
/lrde/home2/stual/stage_DAS/m1_independant_ner_layers
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_1


In [55]:
import glob

ls = sorted(glob.glob(f"{OUT_BASE}/*_metrics/*"))
ls

['/work/stual/res_ICDAR/method_1/m1-110-experiment_1_metrics/111-camembert-ner-multihead-io',
 '/work/stual/res_ICDAR/method_1/m1-110-experiment_1_metrics/112-camembert-ner-multihead-iob2',
 '/work/stual/res_ICDAR/method_1/m1-110-experiment_1_metrics/113-pretrained-camembert-ner-multihead-io',
 '/work/stual/res_ICDAR/method_1/m1-110-experiment_1_metrics/114-pretrained-camembert-multihead-iob2',
 '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/121-camembert-ner-multihead-io',
 '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/122-camembert-ner-multihead-iob2',
 '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/123-pretrained-camembert-ner-multihead-io',
 '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2']

## Paths

In [56]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
FORMAT = "IOB2"
METRICS_OUTPUT_DIR = OUT_BASE / "m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2"
METRICS_OUTPUT_DIR

PosixPath('/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2')

In [57]:
import json
import glob

RUNS = sorted(glob.glob(f"{METRICS_OUTPUT_DIR}/run_*"))
L1_test = []
L2_test = []
for r in RUNS:
    l1 = glob.glob(f"{r}" + "/level-1/test_*")
    L1_test.append(l1[0])
    l2 = glob.glob(f"{r}" + "/level-2/test_*")
    L2_test.append(l2[0])
L1_test,L2_test

(['/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2/run_1/level-1/test_6084.json',
  '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2/run_2/level-1/test_6084.json',
  '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2/run_3/level-1/test_6084.json',
  '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2/run_4/level-1/test_6084.json',
  '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2/run_5/level-1/test_6084.json'],
 ['/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2/run_1/level-2/test_6084.json',
  '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pretrained-camembert-multihead-iob2/run_2/level-2/test_6084.json',
  '/work/stual/res_ICDAR/method_1/m1-120-experiment_2_metrics/124-pr

## Prepare data
Align L1 and L2 predictions and labels

In [58]:
def cleanLists(res):
    entries = []
    for entry in res:
        entry = entry.split(", ")
        ftag = []
        for tag in entry:
            tag = tag.replace("'",'')
            ftag.append(tag)
        entries.append(ftag)
    return entries

In [59]:
results = {} #results[run][set][entree]   set : predictionsl1 labelsl1 predictionsl2 labelsl2

#for each run
for i in range(len(L1_test)):
    # Opening JSON file
    f1 = open(L1_test[i])
    f2 = open(L2_test[i])
    # returns JSON object as
    # a dictionary
    data1 = json.load(f1)
    data2 = json.load(f2)
    
    f1.close()
    f2.close()
    
    predictions1 = data1['eval_predictions'][2:-2]
    predictions1 = predictions1.split('], [')
    labels1 = data1['eval_labels'][2:-2]
    labels1 = labels1.split('], [')
    predictions2 = data2['eval_predictions'][2:-2]
    predictions2 = predictions2.split('], [')
    labels2 = data2['eval_labels'][2:-2]
    labels2 = labels2.split('], [')
    
    fpredictions1 = cleanLists(predictions1)
    fpredictions2 = cleanLists(predictions2)
    flabels1 = cleanLists(labels1)
    flabels2 = cleanLists(labels2)
    
    results[f"run_{i+1}"] = {"gold_l1":flabels1,"gold_l2":flabels2,"predictions_l1":fpredictions1,"predictions_l2":fpredictions2}

In [60]:
results.keys()

dict_keys(['run_1', 'run_2', 'run_3', 'run_4', 'run_5'])

In [61]:
results["run_1"].keys()

dict_keys(['gold_l1', 'gold_l2', 'predictions_l1', 'predictions_l2'])

## Compute joint-labels

In [62]:
def createJointLabels(list1,list2):
    entries = []
    for i,(entryl1,entryl2) in enumerate(zip(list1,list2)):
        assert len(entryl1) == len(entryl2)
        new_tags = []
        for j in range(len(entryl1)):
            tag = entryl1[j] + '+' + entryl2[j]
            tag = tag.replace('+I-','+i_')
            tag = tag.replace('I-','I-i_')
            tag = tag.replace('+B-','+b_')
            tag = tag.replace('B-','I-b_')
            new_tags.append(tag)
        entries.append(new_tags)
    return entries

In [63]:
i = 20
jl_gold = createJointLabels(results["run_1"]["gold_l1"],results["run_1"]["gold_l2"])
jl_preds = createJointLabels(results["run_1"]["predictions_l1"],results["run_1"]["predictions_l2"])
print(jl_gold[i])
print(jl_preds[i])

['I-b_PER+O', 'I-i_PER+O', 'I-i_PER+O', 'I-i_PER+O', 'I-i_PER+O', 'O+O', 'I-b_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'O+O', 'I-b_SPAT+b_LOC', 'I-i_SPAT+O', 'I-i_SPAT+b_CARDINAL', 'O+O']
['I-b_PER+O', 'I-i_PER+O', 'I-i_PER+O', 'I-i_PER+O', 'I-i_PER+O', 'O+O', 'I-b_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'I-i_ACT+O', 'O+O', 'I-b_SPAT+b_LOC', 'I-i_SPAT+O', 'I-i_SPAT+b_CARDINAL', 'O+O']


In [64]:
def createL1L2Labels(list1,list2):
    entries = []
    for i,(entryl1,entryl2) in enumerate(zip(list1,list2)):
        assert len(entryl1) == len(entryl2)
        new_tags = []
        for j in range(len(entryl1)):
            if 'O' in entryl2[j]:
                prefixe_l2 = ''
                tag_l2 = 'O'
            else:
                prefixe_l2, tag_l2 = entryl2[j].split('-')
                
            if 'O' in entryl1[j]:
                prefixe_l1 = ''
                tag_l1 = 'O'
            else:
                prefixe_l1, tag_l1 = entryl1[j].split('-')
                
            if prefixe_l2 == '' and prefixe_l1 != '':
                new_tags.append(prefixe_l1 + '-' + tag_l2 + '+' + tag_l1)
            elif prefixe_l2 == '' and prefixe_l1 == '':
                new_tags.append('O+O')
            else:
                new_tags.append(prefixe_l2 + '-' + tag_l2 + '+' + tag_l1)
                
        entries.append(new_tags)
    return entries

In [65]:
l1l2_gold = createL1L2Labels(results["run_1"]["gold_l1"],results["run_1"]["gold_l2"])
l1l2_preds = createL1L2Labels(results["run_1"]["predictions_l1"],results["run_1"]["predictions_l2"])
print(l1l2_gold[i])
print(l1l2_preds[i])

['B-O+PER', 'I-O+PER', 'I-O+PER', 'I-O+PER', 'I-O+PER', 'O+O', 'B-O+ACT', 'I-O+ACT', 'I-O+ACT', 'I-O+ACT', 'I-O+ACT', 'I-O+ACT', 'O+O', 'B-O+SPAT', 'I-O+SPAT', 'B-CARDINAL+SPAT', 'O+O']
['B-O+PER', 'I-O+PER', 'I-O+PER', 'I-O+PER', 'I-O+PER', 'O+O', 'B-O+ACT', 'I-O+ACT', 'I-O+ACT', 'I-O+ACT', 'I-O+ACT', 'I-O+ACT', 'O+O', 'B-O+SPAT', 'I-O+SPAT', 'B-CARDINAL+SPAT', 'O+O']


In [66]:
def createL1L2Labels(jlentries):
    l1 = []
    l2 = []
    l1_p = []
    l2_p = []
    res = []
    for tags in jlentries:
        new_tags = []
        #print(tags)
        for elem in tags:
            if elem[0] == 'I' and '+O' not in elem:
                parts = elem.split('+')
                e1 = parts[0][4:]
                p1 = parts[0][2:3].upper()
                e2 = parts[1][2:]
                p2 = parts[1][:1].upper()
                tag = p2 + '-' + e2 + '+' + e1
            elif elem[0] == 'I' and elem[-1] == 'O':
                parts = elem.split('+')
                e1 = parts[0][4:]
                p1 = parts[0][2:3].upper()
                e2 = 'O'
                p2 = parts[1][:1].upper()
                tag = p1 + '-' + e2 + '+' + e1
            elif elem[0] == 'O' and len(elem) > 3:
                parts = elem.split('+')
                e1 = 'O'
                p1 = ''
                e2 = parts[1][2:]
                p2 = parts[1][:1].upper()
                tag = p2 + '-' + e2 + '+' + e1
                #print(e1,p1,e2,p2)
            else:
                e1 = 'O'
                p1 = ''
                e2 = 'O'
                p2 = ''
                tag = e2 + '+' + e1
            new_tags.append(tag)
        res.append(new_tags)
    return res

In [67]:
def createFlatLabels(jl_gold,jl_preds,FORMAT):
    
    LABELS_ID_TO_DAS = []
    if FORMAT == 'IO':
        LABELS_ID = {
            "O+O" : 0,
            "I-i_PER+O" : 1,
            "I-i_PER+i_TITREH" : 2,
            "I-i_ACT+O" : 3,
            "I-i_DESC+O" : 4,
            "I-i_DESC+i_ACT" : 5,
            "I-i_DESC+i_TITREP" : 6,
            "I-i_SPAT+O" : 7,
            "I-i_SPAT+i_LOC" : 8,
            "I-i_SPAT+i_CARDINAL" : 9,
            "I-i_SPAT+i_FT" : 10,
            "I-i_TITRE+O" : 11
        }
        LABELS_ID_TO_DAS =["O","I-PER","I-TITRE","I-ACT","O","I-ACT","I-TITRE","O","I-LOC","I-CARDINAL","I-FT","I-TITRE"]
        
    elif FORMAT == "IOB2":
        LABELS_ID = {
            "O+O" : 0,
            "I-b_PER+O" : 1,
            "I-i_PER+O" : 2,
            "I-b_PER+b_TITREH" : 3,
            "I-i_PER+b_TITREH" : 4,
            "I-i_PER+i_TITREH" : 5,
            "I-b_ACT+O" : 6,
            "I-i_ACT+O" : 7,
            "I-b_DESC+O" : 8,
            "I-i_DESC+O" : 9,
            "I-b_DESC+b_ACT" : 10,
            "I-i_DESC+b_ACT" : 11,
            "I-i_DESC+i_ACT" : 12,
            "I-b_DESC+b_TITREP" : 13,
            "I-i_DESC+b_TITREP" : 14,
            "I-i_DESC+i_TITREP" : 15,
            "I-b_SPAT+O" : 16,
            "I-i_SPAT+O" : 17,
            "I-b_SPAT+b_LOC" : 18,
            "I-i_SPAT+b_LOC" : 19,
            "I-i_SPAT+i_LOC" : 20,
            "I-b_SPAT+b_CARDINAL" : 21,
            "I-i_SPAT+b_CARDINAL" : 22,
            "I-i_SPAT+i_CARDINAL" : 23,
            "I-b_SPAT+b_FT" : 24,
            "I-i_SPAT+b_FT" : 25,
            "I-i_SPAT+i_FT" : 26,
            "I-b_TITRE+O" : 27,
            "I-i_TITRE+O" : 28
        }
        LABELS_ID_TO_DAS = ['O','I-PER','I-PER','I-TITRE','I-TITRE','I-TITRE','I-ACT','I-ACT','O','O','I-ACT','I-ACT','I-ACT','I-TITRE','I-TITRE','I-TITRE','O','O','I-LOC','I-LOC','I-LOC','I-CARDINAL','I-CARDINAL','I-CARDINAL','I-FT','I-FT','I-FT','I-TITRE','I-TITRE']
    
    das_labels = []
    for entry in jl_gold:
        das_entry = []
        for elem in entry:
            das_label = LABELS_ID_TO_DAS[LABELS_ID[elem]]
            das_entry.append(das_label)
        das_labels.append(das_entry)
    
    das_predictions = []
    for entry in jl_preds:
        das_entry = []
        for elem in entry:
            try:
                das_label = LABELS_ID_TO_DAS[LABELS_ID[elem]]
                das_entry.append(das_label)
            except:
                das_label = 'I-NO'
                das_entry.append(das_label)
        das_predictions.append(das_entry)
        
    return das_labels, das_predictions

In [68]:
from datasets import load_metric
from multihead_utils.tools import unique

for i in range(0,len(L1_test)):
    r = i+1
    ################ L1
    metric_l1 = load_metric("seqeval")
    results_l1 = metric_l1.compute(predictions=results[f"run_{r}"]["predictions_l1"], references=results[f"run_{r}"]["gold_l1"])
    
    ################ L2
    metric_l2 = load_metric("seqeval")
    results_l2 = metric_l2.compute(predictions=results[f"run_{r}"]["predictions_l2"], references=results[f"run_{r}"]["gold_l2"])
    
    ################ Joint-labels metrics (including prefixes)
    jl_predictions = createJointLabels(results[f"run_{r}"]["predictions_l1"],results[f"run_{r}"]["predictions_l2"]) #Gold
    jl_labels = createJointLabels(results[f"run_{r}"]["gold_l1"],results[f"run_{r}"]["gold_l2"]) #Predictions
    
    metric_jl = load_metric("seqeval")
    results_jl = metric_jl.compute(predictions=jl_predictions, references=jl_labels)
    
    #Save the joint-labels
    #jointlabelslist = unique(jl_predictions)
    
    ################ L1+l2
    if FORMAT == 'IOB2':
        l1l2_predictions = createL1L2Labels(jl_predictions)
        l1l2_labels = createL1L2Labels(jl_labels)
    else:
        l1l2_predictions = jl_predictions
        l1l2_labels = jl_labels
    
    metric_l1l2 = load_metric("seqeval")
    results_l1l2 = metric_l1l2.compute(predictions=l1l2_predictions, references=l1l2_labels)
    
    ################ DAS
    das_labels, das_predictions = createFlatLabels(jl_gold,jl_preds,FORMAT)
    
    metric_das = load_metric("seqeval")
    results_das = metric_das.compute(predictions=das_predictions, references=das_labels)
    
    ################# Global metrics
    all_preds = results[f"run_{r}"]["predictions_l1"] + results[f"run_{r}"]["predictions_l2"]
    all_labels = results[f"run_{r}"]["gold_l1"] + results[f"run_{r}"]["gold_l2"]

    metric_all = load_metric("seqeval")
    results_all = metric_all.compute(predictions=all_preds, references=all_labels)
    
    scores = {
        #Global results (L1 and L2)
        "eval_precision-all": results_all["overall_precision"],
        "eval_recall-all": results_all["overall_recall"],
        "eval_f1-all": results_all["overall_f1"],
        "eval_accuracy-all": results_all["overall_accuracy"],
        #L1
        "eval_precision-l1": results_l1["overall_precision"],
        "eval_recall-l1": results_l1["overall_recall"],
        "eval_f1-l1": results_l1["overall_f1"],
        "eval_accuracy-l1": results_l1["overall_accuracy"],
        #L2
        "eval_precision-l2": results_l2["overall_precision"],
        "eval_recall-l2": results_l2["overall_recall"],
        "eval_f1-l2": results_l2["overall_f1"],
        "eval_accuracy-l2": results_l2["overall_accuracy"],
        #Joint-labels (includign prefixes)
        "eval_precision": results_jl["overall_precision"],
        "eval_recall": results_jl["overall_recall"],
        "eval_f1": results_jl["overall_f1"],
        "eval_accuracy": results_jl["overall_accuracy"],
        #L1+L2
        "eval_precision-l1l2": results_l1l2["overall_precision"],
        "eval_recall-l1l2": results_l1l2["overall_recall"],
        "eval_f1-l1l2": results_l1l2["overall_f1"],
        "eval_accuracy-l1l2": results_l1l2["overall_accuracy"],
        #DAS
        "eval_precision-das": results_das["overall_precision"],
        "eval_recall-das": results_das["overall_recall"],
        "eval_f1-das": results_das["overall_f1"],
        "eval_accuracy-das": results_das["overall_accuracy"],
        #"joint-labels-list":f"{jointlabelslist}",
        #By class
        "eval_PER": results_all['PER'],
        "eval_ACT": results_all['ACT'],
        "eval_ACT_L1": results_l1['ACT'],
        "eval_ACT_L2": results_l2['ACT'],
        "eval_DESC": results_all['DESC'],
        "eval_TITREH": results_all['TITREH'],
        "eval_TITREP": results_all['TITREP'],
        "eval_SPAT": results_all['SPAT'],
        "eval_LOC": results_all['LOC'],
        "eval_CARDINAL": results_all['CARDINAL'],
        "eval_FT": results_all['FT']
    }
    
    #if 'TITRE' in list(results_all.keys()):
        #scores["TITRE"] = f"{results_all['eval_TITRE']}"
    print(scores)
    if not os.path.isdir(f"{METRICS_OUTPUT_DIR}/run_{r}"):
        os.mkdir(f"{METRICS_OUTPUT_DIR}/run_{r}")

    with open(f"{METRICS_OUTPUT_DIR}/run_{r}/test_{str(TRAINSETS_SIZES[0])}.json", 'w') as fp:
        json.dump(scores, fp, default=str)

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+b_TITREH seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_TITREH seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/pytho

{'eval_precision-all': 0.9403165398091096, 'eval_recall-all': 0.9452271071168327, 'eval_f1-all': 0.9427654291078674, 'eval_accuracy-all': 0.9548206921751841, 'eval_precision-l1': 0.9297565179918121, 'eval_recall-l1': 0.9500220167327169, 'eval_f1-l1': 0.9397800283131874, 'eval_accuracy-l1': 0.9460771519548989, 'eval_precision-l2': 0.9537953795379538, 'eval_recall-l2': 0.9393282773564464, 'eval_f1-l2': 0.9465065502183406, 'eval_accuracy-l2': 0.9635642323954691, 'eval_precision': 0.9373561013046815, 'eval_recall': 0.9346661564951215, 'eval_f1': 0.9360091962831688, 'eval_accuracy': 0.9216996398183431, 'eval_precision-l1l2': 0.9340223530825622, 'eval_recall-l1l2': 0.9394415568717515, 'eval_f1-l1l2': 0.9367241171507774, 'eval_accuracy-l1l2': 0.9223782429399175, 'eval_precision-das': 0.9467835341994052, 'eval_recall-das': 0.9441236147963166, 'eval_f1-das': 0.9454517036573928, 'eval_accuracy-das': 0.933601294565955, 'eval_PER': {'precision': 0.947953216374269, 'recall': 0.9620178041543027, 'f1

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_TITREH seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+b_TITREH seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_TITREP seems not to be NE tag.


{'eval_precision-all': 0.9394048909769908, 'eval_recall-all': 0.9470488219577362, 'eval_f1-all': 0.9432113698215906, 'eval_accuracy-all': 0.9609542203894138, 'eval_precision-l1': 0.9348817017581941, 'eval_recall-l1': 0.9482606781153676, 'eval_f1-l1': 0.941523663788392, 'eval_accuracy-l1': 0.9567260009396044, 'eval_precision-l2': 0.9450460205739036, 'eval_recall-l2': 0.945557963163597, 'eval_f1-l2': 0.9453019225561873, 'eval_accuracy-l2': 0.9651824398392232, 'eval_precision': 0.938645038167939, 'eval_recall': 0.9409795293667496, 'eval_f1': 0.9398108340498711, 'eval_accuracy': 0.9355327034504358, 'eval_precision-l1l2': 0.9320214669051878, 'eval_recall-l1l2': 0.9446391877190862, 'eval_f1-l1l2': 0.9382879097130506, 'eval_accuracy-l1l2': 0.9361591063318891, 'eval_precision-das': 0.9467835341994052, 'eval_recall-das': 0.9441236147963166, 'eval_f1-das': 0.9454517036573928, 'eval_accuracy-das': 0.933601294565955, 'eval_PER': {'precision': 0.9580873671782763, 'recall': 0.9632047477744807, 'f1':

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+b_CARDINAL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_ACT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/sit

{'eval_precision-all': 0.9291919552540759, 'eval_recall-all': 0.9482632985183386, 'eval_f1-all': 0.9386307627577086, 'eval_accuracy-all': 0.9536461867724592, 'eval_precision-l1': 0.9284791038345541, 'eval_recall-l1': 0.9489211800968737, 'eval_f1-l1': 0.938588850174216, 'eval_accuracy-l1': 0.9549511927754868, 'eval_precision-l2': 0.9300717894177081, 'eval_recall-l2': 0.9474539544962081, 'eval_f1-l2': 0.9386824097678788, 'eval_accuracy-l2': 0.9523411807694315, 'eval_precision': 0.9281595613952169, 'eval_recall': 0.93925770040176, 'eval_f1': 0.933675652545999, 'eval_accuracy': 0.9205512345356789, 'eval_precision-l1l2': 0.9176678445229682, 'eval_recall-l1l2': 0.9417381844554575, 'eval_f1-l1l2': 0.9295472170852472, 'eval_accuracy-l1l2': 0.9213864383776166, 'eval_precision-das': 0.9467835341994052, 'eval_recall-das': 0.9441236147963166, 'eval_f1-das': 0.9454517036573928, 'eval_accuracy-das': 0.933601294565955, 'eval_PER': {'precision': 0.9662921348314607, 'recall': 0.96973293768546, 'f1': 0.

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+b_TITREH seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+b_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_TITREH seems not to be NE tag.
  w

{'eval_precision-all': 0.940152339499456, 'eval_recall-all': 0.944376973524411, 'eval_f1-all': 0.9422599212359891, 'eval_accuracy-all': 0.9570392023803309, 'eval_precision-l1': 0.9301017095866696, 'eval_recall-l1': 0.9462791721708499, 'eval_f1-l1': 0.9381207028265852, 'eval_accuracy-l1': 0.9508273738059195, 'eval_precision-l2': 0.9528767123287671, 'eval_recall-l2': 0.9420368364030336, 'eval_f1-l2': 0.9474257695450831, 'eval_accuracy-l2': 0.9632510309547424, 'eval_precision': 0.9405826362849726, 'eval_recall': 0.9358140424717811, 'eval_f1': 0.9381922800287702, 'eval_accuracy': 0.9258756590280315, 'eval_precision-l1l2': 0.9339259971167708, 'eval_recall-l1l2': 0.9396833071437205, 'eval_f1-l1l2': 0.936795806471049, 'eval_accuracy-l1l2': 0.9265020619094848, 'eval_precision-das': 0.9467835341994052, 'eval_recall-das': 0.9441236147963166, 'eval_f1-das': 0.9454517036573928, 'eval_accuracy-das': 0.933601294565955, 'eval_PER': {'precision': 0.9621972829297105, 'recall': 0.9667655786350149, 'f1':

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+b_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+i_TITREH seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-

{'eval_precision-all': 0.9294412607449857, 'eval_recall-all': 0.9454700024289531, 'eval_f1-all': 0.9373871161950632, 'eval_accuracy-all': 0.9566216004593621, 'eval_precision-l1': 0.9173819742489271, 'eval_recall-l1': 0.9412153236459709, 'eval_f1-l1': 0.9291458378613344, 'eval_accuracy-l1': 0.9462337526752623, 'eval_precision-l2': 0.9445640473627557, 'eval_recall-l2': 0.9507042253521126, 'eval_f1-l2': 0.9476241900647948, 'eval_accuracy-l2': 0.9670094482434619, 'eval_precision': 0.9356251785544234, 'eval_recall': 0.93983164339009, 'eval_f1': 0.9377236936292054, 'eval_accuracy': 0.9278592681526335, 'eval_precision-l1l2': 0.922559320033054, 'eval_recall-l1l2': 0.9446391877190862, 'eval_f1-l1l2': 0.9334687052078356, 'eval_accuracy-l1l2': 0.9290076734352978, 'eval_precision-das': 0.9467835341994052, 'eval_recall-das': 0.9441236147963166, 'eval_f1-das': 0.9454517036573928, 'eval_accuracy-das': 0.933601294565955, 'eval_PER': {'precision': 0.9471520845566647, 'recall': 0.9572700296735905, 'f1':